In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
from drive import segment_points
from tip_finding import tip_finding
from collections import deque

In [11]:
from collections import deque
#use append and pop functions (acts LIFO)
stack = deque()
root_id = 864691135247440303
good_tips_thick, good_tips_thin, good_tips_bad_thick, good_tips_bad_thin, just_tips, just_means  = tip_finding.endpoints_from_rid(root_id)
#our list of all endpoints to extend
endpoints = np.concatenate([good_tips_thick, good_tips_thin])
#OBTAIN ACTUAL THRESHOLD NUMBER
threshold = 0.3\
#this is for one SEG_ID--PUT INTO FUNCTION; run this for each seg_ID
for tip in endpoints:
    #reset stack everytime we move on to extending a new tip(endpoint)
    next_seg_id = tip
    while(max_confidence > threshold and next_seg_id.num_soma == (0)):
        stack.append(next_seg_id)
        ext, num = segment_points(endpoint=stack.pop(), root_id=root_id, point_id=0, resolution=[8,8,40])
        #need to loop through values to get largest confidence
        val_list = list(ext.merges.values())
        key_list = list(ext.merges.keys())
        max_confidence = max(ext.merges.values())
        position = val_list.index(max_confidence)
        #obtain the extension associated with highest confidence
        next_seg_id = key_list[position]
        #stack.append(next_seg_id)
        #make sure this next seg_ID is an orphan and has enough confidence



    


branch


100%|██████████| 21334/21334 [00:00<00:00, 268041.25it/s]
/Users/sheeltanna/campfire-1/tip_finding/tip_finding.py:516: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  locs = np.array([np.array(list(nx.get_node_attributes(g, 'mean_loc').values())) for g in graphs])
100%|██████████| 14/14 [00:00<00:00, 628.76it/s]
2it [00:00, 367.92it/s]
1it [00:00, 225.74it/s]


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
#VERSION 2
from collections import deque
#use append and pop functions (acts LIFO)
stack = deque()
root_id = 864691135247440303
good_tips_thick, good_tips_thin, good_tips_bad_thick, good_tips_bad_thin, just_tips, just_means  = tip_finding.endpoints_from_rid(root_id)
#our list of all endpoints to extend
endpoints = np.concatenate([good_tips_thick, good_tips_thin])
#OBTAIN ACTUAL THRESHOLD NUMBER
threshold = 0.3
#this is for one SEG_ID--PUT INTO FUNCTION; run this for each seg_ID
 #add all endpoints in stack
for tip in endpoints:
    #reset stack everytime we move on to extending a new tip(endpoint)
    #TUPLE: (SEG_ID1, tip1), (SEG_ID1, tip2)
    stack.append((tip, root_id))
while(stack):
    next_seg_id_endpoint = stack.pop()
    ext, num = segment_points(endpoint=next_seg_id_endpoint[0], root_id=next_seg_id_endpoint[1], point_id=0, resolution=[8,8,40])
    #need to loop through values to get largest confidence
    val_list = list(ext.merges.values())
    key_list = list(ext.merges.keys())
    max_confidence = max(ext.merges.values())
    position = val_list.index(max_confidence)
    #obtain the extension associated with highest confidence
    next_seg_id = key_list[position]
    if (max_confidence > threshold and next_seg_id.num_soma == (0)):
        #get endpoints for next seg_ID
        good_tips_thick, good_tips_thin, good_tips_bad_thick, good_tips_bad_thin, just_tips, 
        just_means = tip_finding.endpoints_from_rid(next_seg_id)
        endpoints = np.concatenate([good_tips_thick, good_tips_thin])
        #append all of the endpoints
        for tip in endpoints:
            #reset stack everytime we move on to extending a new tip(endpoint)
            #TUPLE: (SEG_ID1, tip1), (SEG_ID1, tip2)
            stack.append((tip, next_seg_id))

### Implementation of pseudocode from ipad

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
from drive import segment_points
from tip_finding import tip_finding
from collections import deque

In [149]:
def append_endpoints_to_stack(extension_stack, root_id, ext_ids):
    #root_id = root_id
    # Find endpoints
    good_tips_thick, good_tips_thin, good_tips_bad_thick, good_tips_bad_thin, just_tips, just_means = tip_finding.endpoints_from_rid(root_id)
    endpoints = []
    # if seg_id is not in ext_ids, then add it to ext_ids
    if ((root_id in ext_ids["seg_id"].unique()) == False):
        #adding to end
        ext_ids.loc[len(ext_ids.index)] = [root_id, 0, {}]

        # FIND A BETTER WAY TO DO THIS
        for i in good_tips_thick:
            extension_stack.append((root_id, i))
            endpoints.append(i)
        for i in good_tips_thin:
            extension_stack.append((root_id, i))
            endpoints.append(i)
        
        endpoints_ext_dict = {}
        for i in endpoints:
            endpoints_ext_dict[tuple(i.tolist())] = 0
            # endpoints_ext_dict[tuple(i)] = 0
            #already a list right?
        
        #create vector or list of each collumn: one vector is all extensions the other is all endpoints but must match
        #make them into 2 seperate collumns 
        #isn't ep_and_next_seg a tuple of dict,next_seg_id? so how are you just adding the dict directly without slicing? 
        ext_ids.loc[ext_ids['seg_id'] == root_id,
                    'ep_and_next_seg'] = [endpoints_ext_dict]
        


    
    # if seg_id is in ext_ids, check if it is fully extended. If it is not fully extended, add the non-extended points to the extension_stack
    # Theoretically should never reach here because if we hit a seg_id, it should either be fully extended or not, but never in between, but this is here just in case ig <3 ...?
    # Have not tested this!
    else:
        if (check_fully_extended(ext_ids = ext_ids, root_id = root_id) == False):
            dict_of_endpoints_and_extensions = ext_ids.loc[ext_ids.seg_id == root_id].ep_and_next_seg[0]
            # find endpoints in dict with next_seg_ids = 0 and add these to the extension_stack
            endpoints = {i for i in dict_of_endpoints_and_extensions if dict_of_endpoints_and_extensions[i] == 0}
            
            # Push (root_id, endpoint) onto stack for each un-extended endpoint
            for i in endpoints:
                extension_stack.append((root_id, i))

    return extension_stack, ext_ids

In [9]:
def get_highest_confidence(ext):
    # use ext to extrace highest confidence merge and associated confidence

    #example:
    highest_confidence_merge = 864691134406233920
    highest_confidence = 0.8
    
    #implementation:
    #need to loop through values to get largest confidence
    val_list = list(ext.merges.values())
    key_list = list(ext.merges.keys())
    #index of highest value
    highest_confidence = np.max(val_list)
    highest_confidence_position = np.argmax(val_list)
    #position = val_list.index(highest_confidence)
    #obtain the extension associated with highest confidence
    highest_confidence_merge = key_list[highest_confidence_position]
    
    return highest_confidence_merge, highest_confidence

In [151]:
def check_fully_extended(ext_ids, root_id):
    return 0 in ext_ids.loc[ext_ids.seg_id == root_id].ep_and_next_seg[0].values()
    

In [152]:
# Initializing all necessary values
#creating our dataframe
ext_ids = pd.DataFrame(columns=["seg_id", "fully_extended", "ep_and_next_seg"])
extension_stack = deque()
min_confidence = 0.8

In [146]:
# Actually interacting with stack
first_root_id = 864691135247440303
extension_stack, ext_ids = append_endpoints_to_stack(extension_stack = extension_stack, root_id = first_root_id, ext_ids = ext_ids)

while(len(extension_stack) != 0):
#while(extension_stack)
    #breaks up tuple for you?
    curr_root_id, curr_endpoint = extension_stack.pop()
    
    # if this seg_id has been fully extended, skip it
    if (int(ext_ids.loc[ext_ids.seg_id == curr_root_id].fully_extended) == 1):
        continue

    else:
        ext, num = segment_points(endpoint = curr_endpoint, root_id = curr_root_id, point_id = 0, resolution = [8,8,40])
        #ext = 0
        next_seg, highest_confidence = get_highest_confidence(ext)
        if (highest_confidence >= min_confidence):
            # Updating the next_seg_id for the current endpoint in the ext_ids dataFrame
            #updating the dictionary
            ext_ids.loc[ext_ids.seg_id == curr_root_id].ep_and_next_seg[0][tuple(curr_endpoint.tolist())] = next_seg

            # checking if curr_seg id is fully extended, if yes, update it in ext_ids
            if (check_fully_extended(ext_ids = ext_ids, root_id = curr_root_id)):
                ext_ids.loc[ext_ids.seg_id == curr_root_id].fully_extended = 1
            
            # If the next_seg is not fully extended, add it's endpoints to the stack
            # This if statement is the only one that i haven't tested yet
            # Avoids pushing endpoints in next_seg that are already segmented in append_endpoints_to_stack() - although this should theroetically never happen.
            if ((next_seg not in ext_ids['seg_id'].unique()) or (int(ext_ids.loc[ext_ids.seg_id == next_seg].fully_extended) == 0)):
                extension_stack, ext_ids = append_endpoints_to_stack(extension_stack = extension_stack, root_id = next_seg, ext_ids = ext_ids)

        else:
            ext_ids.loc[ext_ids.seg_id == curr_root_id].ep_and_next_seg = 1

branch


100%|██████████| 21334/21334 [00:00<00:00, 101300.06it/s]
/Users/RupaChalavadi/Desktop/CRIMSON/Summer/Campfire/campfire/tip_finding/tip_finding.py:516: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  locs = np.array([np.array(list(nx.get_node_attributes(g, 'mean_loc').values())) for g in graphs])
100%|██████████| 14/14 [00:00<00:00, 292.15it/s]
2it [00:00, 95.12it/s]
1it [00:00, 64.59it/s]


In [3]:
root_id = 864691135247440303
    # Find endpoints
good_tips_thick, good_tips_thin, good_tips_bad_thick, good_tips_bad_thin, just_tips, just_means = tip_finding.endpoints_from_rid(root_id)
endpoints = []
for i in good_tips_thick:
    endpoints.append(i)
    for i in good_tips_thin:
        endpoints.append(i)

branch


100%|██████████| 21334/21334 [00:00<00:00, 332310.62it/s]
/Users/sheeltanna/campfire-1/tip_finding/tip_finding.py:516: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  locs = np.array([np.array(list(nx.get_node_attributes(g, 'mean_loc').values())) for g in graphs])
100%|██████████| 14/14 [00:00<00:00, 646.73it/s]
3it [00:00, 339.90it/s]
1it [00:00, 195.89it/s]

[1620072.11650485  906418.7184466   944748.        ]
[1635877.40909091  879300.86363636  955709.04545455]


In [8]:

endpoints_ext_dict = {}
for i in endpoints:
    # print(i)
    # print()
    endpoints_ext_dict[tuple(i)] = 0

for i in endpoints_ext_dict.keys():
    print(i)

(1620072.1165048543, 906418.718446602, 944748.0)
(1635877.4090909092, 879300.8636363636, 955709.0454545454)
